---

# Logistic Regression

By: Laura Moses 

--- 

## Definition
**Logistic Regression** is a supervised learning classification algorithm used to predict the probability of a target variable $y$. The nature of target or dependent variable is dichotomous, meaning there are only two possible classes. Thus, the target is binary, and represents either `1`, a success/yes/win, or `0`, a failure/no/loss. 

Mathematically, a logistic regression model predicts $P(Y=1)$ as a function of $X$. It is one of the simplest machine learning algorithms that can be used for various classification problems that involve detection.

---

### Assumptions

* The target variable is binary, with the desired outcome represented by the factor level 1
* There is no multi-collinearity present in the model
* Model variables are meaningful
* Large sample size 

---

### Advantages 

* Simple algorithm
* Useful for predicting 

---

### Disadvantages

* Adding independent variables to model can result in overfitting
* R^2^ can have computation issues causing them to be artifically high or low, so must be mindful when interpreting goodness of fit

---

## Model

In this notebook, we will be using the dataset [SAheart](https://web.stanford.edu/~hastie/ElemStatLearn//datasets/SAheart.data), available within this repository, to predict if a person has coronary heart disease (CHD) based on some attributes.

The dataset comes from a retrospective sample of males in a heart-disease high-risk region of the Western Cape, South Africa. There are roughly two controls per case of CHD, thus this is our response $y$. These data are taken from a larger dataset, described in  *Rousseauw et al, 1983, South African Medical Journal*. 

The dataset includes the following columns: 

* `sbp` = Systolic blood pressure
* `tobacco`	= Cumulative tobacco use (kg)
* `ldl` = Low densiity lipoprotein cholesterol
* `adiposity`
* `famhist` = Family history of heart disease (Present, Absent)
* `typea` = Type-A behavior
* `obesity`
* `alcohol` = Current alcohol consumption
* `age`	= Age at onset
* `chd` = *Target*, coronary heart disease (1, 0)

---

### Method

We want to predict whether or not a male has coronary heart disease (`chd`) based on three features: `sbp`, `ldl`, and `alcohol`.

**<ins>Feed Forward</ins>** (with a single neuron)
$$
(x^1, y^1),\cdots,(x^N, y^N); \space \space
x^i =  
 \left[{\begin{array}{c}
male \  i \  sys \  bp \\
male \  i \  ldl \  chol \\
male \  i \  alcohol
 \end{array}}\right] \\
y^i \in \{0,1\} \\
where \  1 \  indicates \  male \  i \  had \  CHD, \  0 \  otherwise
 $$
 
 $$Z^i = w^T \cdot x^i + b$$
 
---

**<ins>Loss Function</ins>**

We want $L(\hat {y^i}, y^i)=$ How close $\hat{y^i}$ is to $y^i$!

First consider maximizing $P(y^i|x^i)$, the probability that $\hat{y^i}$ predicts $y^i$. Since there are two discrete outputs, this is subject to the following formula by Bernoulli:

Maximize $\rightarrow P(y^i|x^i) = \hat y^y (1-\hat y)^{1-y}$

**<ins>Decision Boundary</ins>**
Prediction is $1$ if $\hat {y^i} \ge 0.5$, $0$ otherwise.

---

The following packages will be needed to run the code below: 

* CSV [documentation](https://csv.juliadata.org/stable/)
* DataFrames [documentation](https://dataframes.juliadata.org/stable/)

---

In [1]:
using CSV
using DataFrames

data = CSV.read("SAheart.csv", DataFrame)

,row.names,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol
,Int64,Int64,Float64,Float64,Float64,String,Int64,Float64,Float64
1,1,160,12.0,5.73,23.11,Present,49,25.3,97.2
2,2,144,0.01,4.41,28.61,Absent,55,28.87,2.06
3,3,118,0.08,3.48,32.28,Present,52,29.14,3.81
4,4,170,7.5,6.41,38.03,Present,51,31.99,24.26
5,5,134,13.6,3.5,27.78,Present,60,25.99,57.34
6,6,132,6.2,6.47,36.21,Present,62,30.77,14.14
7,7,142,4.05,3.38,16.2,Absent,59,20.81,2.62
8,8,114,4.08,4.59,14.6,Present,62,23.11,6.72
9,9,114,0.0,3.83,19.4,Present,49,24.86,2.49


In [2]:
x_data = [[x[1], x[2], x[3]] for x in zip(data.sbp, data.ldl, data.alcohol)]
y_data = [x for x in data.chd];

---

Below we will define a sigmoid activation function to map predictions to probabilities using the following: 

$$\sigma(z) = \frac{1}{1+e^{-z^i}}$$

where $\sigma(z)$ is an output between $0$ and $1$ (probability estimate), $z^i$ is the algorithms prediction ($wx^i + b$), and $e$ is Euler's number. 

We will use stochastic gradient descent to update our weights and bias.

---

In [3]:
# Sigmoid activation function
σ(x) = 1/(1+exp(-x))

# Loss function
function cross_entropy_loss(x, y, w, b)
    return -y*log(σ(w'x + b)) - (1-y)*log(1 - σ(w'x+b))
end

# Cost function
function average_cost(features, labels, w, b)
    N = length(features)
    return (1/N)*sum([cross_entropy_loss(features[i], labels[i], w, b) for i = 1:N])
end;

In [4]:
## DELETE AND REPLACE WITH STOCHASTIC?
function batch_gradient_descent(features, labels, w, b, α)
    del_w = [0.0 for i = 1:length(w)]
    del_b = 0.0
    
    N = length(features)
    
    for i = 1:N
        del_w += (σ(w'features[i] + b) - labels[i]) * features[i]
        del_b += (σ(w'features[i] + b) - labels[i])
    end
    
    w = w - α*del_w
    b = b - α*del_b
    
    return w, b
end;

---

Now we will test our functions to make sure the cost is decreasing with each iteration.

---

In [5]:
#### CHANGE TO STOACHISTIC
# Initialize weights and bias, choose alpha
w = [0.0, 0.0, 0.0]
b = 0.0
alpha = 0.0000000001

println("The initial cost is: ", average_cost(x_data, y_data, w, b))

for i in 1:4
    w, b = batch_gradient_descent(x_data, y_data, w, b, alpha)
    println("The new cost is: ", average_cost(x_data, y_data, w, b))
end

The initial cost is: 0.6931471805599451
The new cost is: 0.6931296464273247
The new cost is: 0.6931121203412501
The new cost is: 0.6930946022980257
The new cost is: 0.6930770922939568


---

Now that we've verified our stochastic gradient descent function is working, we can define a training stochastic gradient descent function below: 

---

In [6]:
function train_batch_gradient_descent(features, labels, w, b, α, epochs)
    for i = 1:epochs
        w, b = batch_gradient_descent(features, labels, w, b, α)
        
        if i == 1
            println("Epoch ", i, " with cost: ", average_cost(features, labels, w, b))
        end
        
        if i == 100
            println("Epoch ", i, " with cost: ", average_cost(features, labels, w, b))
        end
        
        if i == 1000
            println("Epoch ", i, " with cost: ", average_cost(features, labels, w, b))
        end
        
        if i == 10000
            println("Epoch ", i, " with cost: ", average_cost(features, labels, w, b))
        end        
        
        if i == 100000
            println("Epoch ", i, " with cost: ", average_cost(features, labels, w, b))
        end
    end
    return w, b
end;

In [20]:
### DELETE -- Modified to not print output

function train_batch_gradient_descent(features, labels, w, b, α, epochs)
    for i = 1:epochs
        w, b = batch_gradient_descent(features, labels, w, b, α)
    end
    println("Cost: ", average_cost(features, labels, w, b))
    return w, b
end;

In [22]:
function stochastic_gradient_descent(x_data, y_data, w, b, α)
    
    N = length(x_data)
    
    # pick a random i
    i = rand([k for k = 1:N])
    
    # calculate the gradient only at randomly picked point
    w = w - (-2/N)*α*x_data[i]*(y_data[i] - (w*x_data[i] + b))
    b = b - (-2/N)*α*(y_data[i] - (w*x_data[i] + b))
    
    return w, b
end

function stochastic_train(x_data, y_data, w, b, α, epochs)
    for i = 1:epochs
        w, b = batch_gradient_descent(x_data, y_data, w, b, α)
    end
    println(average_cost(x_data, y_data, w, b))
    return w, b
end;

In [7]:
### EDIT THIS TO HAVE STOCHASTIC
# Initializing 0 for all weights and bias
w = [0.0, 0.0, 0.0]
b = 0.0
alpha = 0.0000000001

w, b = train_batch_gradient_descent(x_data, y_data, w, b, alpha, 10000)

Epoch 1 with cost: 0.6483759045108881
Epoch 100 with cost: 0.648363893476474
Epoch 1000 with cost: 0.6482748653761067
Epoch 10000 with cost: 0.6480785925985565


([-0.00552190290970136, 0.029073227126353927, 0.003927708039620779], -0.00021305501678453012)

---

While training our $w$ and $b$, the goal is to get our cost as close to $0.5$ as possible, since this is our decision boundary. 

---

In [9]:
w, b = train_batch_gradient_descent(x_data, y_data, w, b, 0000000001, 1000000)

Epoch 1 with cost: 0.6469123174328647
Epoch 100 with cost: 0.6469122075870735
Epoch 1000 with cost: 0.6469112090345196
Epoch 10000 with cost: 0.646901228021375
Epoch 100000 with cost: 0.6468018677642113


([-0.00604427938642357, 0.04330469167940441, 0.003947580425113944], -0.0017567425450463032)

In [10]:
w, b = train_batch_gradient_descent(x_data, y_data, w, b, alpha, 1000000)

Epoch 1 with cost: 0.6458519141416657
Epoch 100 with cost: 0.6458518138824914
Epoch 1000 with cost: 0.6458509024770221
Epoch 10000 with cost: 0.6458417925404434
Epoch 100000 with cost: 0.6457511037292287


([-0.006271313325661864, 0.04994181753659419, 0.003985229555625332], -0.002533610719213252)

---

We can see that our cost was decreasing each time we ran the training batch descent. Now, we continue to re-run the code cell below until we reach approximately $0.50$.

---

In [18]:
w, b = train_batch_gradient_descent(x_data, y_data, w, b, alpha, epochs)

Epoch 1 with cost: 0.644878495686086
Epoch 100 with cost: 0.6448784955946222
Epoch 1000 with cost: 0.6448784947631326
Epoch 10000 with cost: 0.644878486448249
Epoch 100000 with cost: 0.6448784032997599


([-0.006272865982679547, 0.04998720227642257, 0.003985487896146975], -0.002539060225144372)

In [ ]:
while average_cost(x_data, y_data, w, b) >= .60
    w, b = stochastic_train(x_data, y_data, w, b, alpha, epochs)
end

0.6448701842088637
0.6448692611324501
0.6448683381402383


In [ ]:
function predict(x, y, w, b)
    if σ(w'x+b) >= .5
        println("Predict Accepted")
        y == 1 ? println("Was Accepted") : println("Was Not Accepted")
    else
        println("Predict Not Accepted")
        y == 1 ? println("Was Accepted") : println("Was Not Accepted")
        
    end
end

In [ ]:
for i = 1:length(x_data)
    predict(x_data[i], y_data[i], w, b)
    println()
end

In [ ]:
function new_predict(x, y, w, b)
    if σ(w'x+b) >= .5
        return 1
    else
        return 0
    end
end

In [ ]:
mean_error = 0.0
for i = 1:length(x_data)
    mean_error += (new_predict(x_data[i], y_data[i], w, b) - y_data[i])^2
end

println(mean_error/length(x_data))